<a href="https://colab.research.google.com/github/nitinverma78/personal-health-insights-agent/blob/main/TryPHIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# 1. Clone the repository
!git clone https://github.com/nitinverma78/personal-health-insights-agent.git

# 2. Move into the directory so Python can find the files
%cd personal-health-insights-agent

# 3. Install dependencies (The repo might miss a requirements.txt, so let's force install the core ones)
!pip install langchain langchain-openai tavily-python pandas python-dotenv
# 4. Uncomment the following line and execute the cell to install OneTwo (if not already installed).
!pip install git+https://github.com/google-deepmind/onetwo

Cloning into 'personal-health-insights-agent'...
remote: Enumerating objects: 346, done.
remote: Counting objects: 100% (346/346), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 346 (delta 116), reused 315 (delta 93), pack-reused 0 (from 0)
Receiving objects: 100% (346/346), 28.56 MiB | 16.08 MiB/s, done.
Resolving deltas: 100% (116/116), done.
Updating files: 100% (263/263), done.
/content/personal-health-insights-agent/personal-health-insights-agent/personal-health-insights-agent/personal-health-insights-agent
  Cloning https://github.com/google-deepmind/onetwo to /tmp/pip-req-build-l9tei4vh
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/onetwo /tmp/pip-req-build-l9tei4vh
  Resolved https://github.com/google-deepmind/onetwo to commit 3cef22fafd10afc06a0b471ae2e30cbd53916595
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning h

In [43]:
import os
from google.colab import userdata

# Load keys safely from Colab Secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("Keys loaded successfully!")

Keys loaded successfully!


In [44]:
# Cell 1: Imports
# Note: the first run of this cell may take several minutes to complete

import os
import sys
import importlib
import glob
from IPython.display import HTML, display
import pandas as pd
import google.generativeai as genai
import ipywidgets

current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

!pip install git+https://github.com/google-deepmind/onetwo

from onetwo import ot
from onetwo.backends import gemini_api

# Import local modules
try:
    import data_utils
    import colab_utils
    import prompt_templates
    import phia_agent

    importlib.reload(data_utils)
    importlib.reload(colab_utils)
    importlib.reload(prompt_templates)
    importlib.reload(phia_agent)
    print("Modules imported/reloaded.")
except ImportError as e:
    print(f"Error importing local modules: {e}")
    print("Please ensure data_utils.py, colab_utils.py, prompt_templates.py, and phia_agent.py are in the same directory as the notebook.")

from data_utils import load_persona
from colab_utils import format_react_state_html
from phia_agent import get_react_agent, QUESTION_PREFIX

  Cloning https://github.com/google-deepmind/onetwo to /tmp/pip-req-build-1ovsw44c
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/onetwo /tmp/pip-req-build-1ovsw44c
  Resolved https://github.com/google-deepmind/onetwo to commit 3cef22fafd10afc06a0b471ae2e30cbd53916595
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/google-deepmind/gemma.git to /tmp/pip-install-5_t9gkuk/gemma_34af4d5f27db4a40a10b6fe120b7daf8
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/gemma.git /tmp/pip-install-5_t9gkuk/gemma_34af4d5f27db4a40a10b6fe120b7daf8
  Resolved https://github.com/google-deepmind/gemma.git to commit b38d84803b7336aed68a3e1c2c930c20df62a4e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Modules imported/reload

In [45]:
# Cell 2: Set API Keys

# Replace with your API keys below.
#
# WARNING: Don't leave your API keys here when committing
# this notebook into any public code repos!
google_api_key = os.environ["GOOGLE_API_KEY"]
tavily_api_key = os.environ["TAVILY_API_KEY"]

# Set the Google API key below and when setting up backend,
# the tavily API key will be set when creating the agent.
genai.configure(api_key=google_api_key)

In [46]:
# Cell 3: Setup LLM Backend

llm_engine = gemini_api.GeminiAPI(
    generate_model_name="models/gemini-2.5-pro", # Can replace with a better model, if desired.
                                                 # Note: due to the nature of our few_shots
                                                 # examples that were designed with Gemini 1.5
                                                 # models in mind, agent behavior with Gemini
                                                 # 2.0, 2.5, 3.0 and beyond models may have errors
                                                 # related to output formatting. Brief testing with
                                                 # this notebook's new default (2.5 Pro) confirms
                                                 # that PHIA's functionality seems to be intact.
                                                 #
                                                 # All Gemini 1.5 models were deprecated alongside
                                                 # the release of Gemini 3.0 models.
    api_key=google_api_key,
    temperature=0.0,
)
llm_engine.register()
print("LLM Backend Registered.")

LLM Backend Registered.


In [47]:
# Cell 4: Load Data

summary_path = os.path.join("synthetic_wearable_users", "summary_df_502.csv")
activities_path = os.path.join("synthetic_wearable_users", "exercise_df_502.csv")

print(f"Loading data from: {summary_path} and {activities_path}")

try:
    if not os.path.exists(summary_path):
        raise FileNotFoundError(f"Cannot find summary file: {summary_path}")
    if not os.path.exists(activities_path):
        raise FileNotFoundError(f"Cannot find activities file: {activities_path}")

    summary_df, activities_df, profile_df = load_persona(
        summary_path=summary_path,
        activities_path=activities_path,
        enforce_schema=True,
        temporally_localize="today"
    )
    print("Data Loaded Successfully:")
    display(f"Summary DF: {summary_df.shape}")
    display(summary_df.head(1))
    display(f"Activities DF: {activities_df.shape}")
    display(activities_df.head(1))
    display(f"Profile DF: {profile_df.shape}")
    display(profile_df.head(1))
except Exception as e:
    print(f"Error loading data: {e}")

Loading data from: synthetic_wearable_users/summary_df_502.csv and synthetic_wearable_users/exercise_df_502.csv
Data Loaded Successfully:


'Summary DF: (29, 20)'

,datetime,resting_heart_rate,heart_rate_variability,fatburn_active_zone_minutes,cardio_active_zone_minutes,peak_active_zone_minutes,active_zone_minutes,steps,rem_sleep_minutes,deep_sleep_minutes,awake_minutes,light_sleep_minutes,sleep_minutes,bed_time,wake_up_time,stress_management_score,deep_sleep_percent,rem_sleep_percent,awake_percent,light_sleep_percent
datetime,,,,,,,,,,,,,,,,,,,,
2025-11-27,2025-11-27,67.77,7.11,17.59,14.81,1.11,33.51,12637.39,61.79,85.32,49.59,194.07,390.77,2025-11-27 21:56:34,2025-11-28 04:27:20,81.46,21.83,15.81,12.69,49.66


'Activities DF: (4, 11)'

,startTime,endTime,activityName,distance,duration,elevationGain,averageHeartRate,calories,steps,activeZoneMinutes,speed
startTime,,,,,,,,,,,
2025-12-09 09:06:00,2025-12-09 09:06:00,2025-12-09 09:32:00,Outdoor Bike,0.0,25.62,30.48,90.0,136.0,0.0,0.0,0.0


'Profile DF: (1, 6)'

,age,gender,averageDailySteps,elderly,height_cm,weight_kg
0,52,Male,10121,No,184,81


In [48]:
# Cell 5: Load Exemplars

exemplar_dir = "few_shots"
exemplar_pattern = os.path.join(exemplar_dir, "*.ipynb")
final_exemplar_paths = glob.glob(exemplar_pattern)

if not final_exemplar_paths:
    print(f"Warning: No exemplar notebooks found in {exemplar_dir}/")
else:
    print(f"Found {len(final_exemplar_paths)} exemplar notebooks:")
    for path in final_exemplar_paths:
        print(f"  - {path}")

Found 46 exemplar notebooks:
  - few_shots/overall_health.ipynb
  - few_shots/bedtime_consistency.ipynb
  - few_shots/strength.ipynb
  - few_shots/cardiovascular_health.ipynb
  - few_shots/sleep_stages_resting_heart_rate.ipynb
  - few_shots/workouts_specific_goals.ipynb
  - few_shots/muscle_recovery_and_injury_prevention.ipynb
  - few_shots/manage_stress.ipynb
  - few_shots/average_HRV_conditioned_by_biking.ipynb
  - few_shots/running_compared_olympian.ipynb
  - few_shots/sleep_data_patterns.ipynb
  - few_shots/exercise_effects_on_sleep.ipynb
  - few_shots/10k_run_goal.ipynb
  - few_shots/highest_rhr_last_30_days.ipynb
  - few_shots/HRV_bedtime.ipynb
  - few_shots/average_HR_and_calorie_burn_activities.ipynb
  - few_shots/mindfulness_effects.ipynb
  - few_shots/strengthen_core.ipynb
  - few_shots/worst_HRV_last_month.ipynb
  - few_shots/sleep_time_after_last_bike_ride.ipynb
  - few_shots/best_activities_to_manage_stress_score.ipynb
  - few_shots/sleep_fastest_run.ipynb
  - few_shots/ma

In [49]:
# Cell 6: Create Agent

try:
    agent = get_react_agent(
        summary_df=summary_df,
        activities_df=activities_df,
        profile_df=profile_df,
        example_files=final_exemplar_paths,
        tavily_api_key=tavily_api_key,
        use_mock_search=False
    )
    print("Agent Created Successfully!")
except Exception as e:
    print(f"Error creating agent: {e}")

Processing file: few_shots/overall_health.ipynb
Processing file: few_shots/bedtime_consistency.ipynb
Processing file: few_shots/strength.ipynb
Processing file: few_shots/cardiovascular_health.ipynb
Processing file: few_shots/sleep_stages_resting_heart_rate.ipynb
Processing file: few_shots/workouts_specific_goals.ipynb
Processing file: few_shots/muscle_recovery_and_injury_prevention.ipynb
Processing file: few_shots/manage_stress.ipynb
Processing file: few_shots/average_HRV_conditioned_by_biking.ipynb
Processing file: few_shots/running_compared_olympian.ipynb
Processing file: few_shots/sleep_data_patterns.ipynb
Processing file: few_shots/exercise_effects_on_sleep.ipynb
Processing file: few_shots/10k_run_goal.ipynb
Processing file: few_shots/highest_rhr_last_30_days.ipynb
Processing file: few_shots/HRV_bedtime.ipynb
Processing file: few_shots/average_HR_and_calorie_burn_activities.ipynb
Processing file: few_shots/mindfulness_effects.ipynb
Processing file: few_shots/strengthen_core.ipynb
P

In [56]:
# Cell 7: Ask a Question

# Define your question below
#question = "How can I feel more awake and energetic during the day?"
#question = "Do I need to go a see a doctor?"
#question = "How can I feel less anxious?"
#question = "How to stop coughing?"
question = "Why do I keep fainting irregularly, especially after long international flights or lack of sleep?"

full_question = QUESTION_PREFIX + question

try:
    final_answer, final_state = ot.run(
        agent(inputs=full_question, return_final_state=True)
    )

    print("\n--- Agent Trace ---")
    html_output = format_react_state_html(final_state)
    display(HTML(html_output))

    print("\n--- Final Answer ---")
    if final_answer:
        display(HTML(str(final_answer).replace('\n', '<br>')))
    else:
        display("No final answer was returned by the agent.")

except Exception as e:
    print(f"An error occurred during agent execution: {e}")


--- Agent Trace ---



--- Final Answer ---
